In [1]:
import time
import numpy as np
import pandas as pd

from Case import *
pd.set_option('display.max_columns',None)
mimic = pd.read_csv("/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Data preproc/mimic_df_cp.csv.gz", compression='gzip')
eicu = pd.read_csv("/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Data preproc/eicu_df_cp.csv.gz", compression='gzip')

mimic_v1, eicu_v1 = filter_cohort(mimic, eicu) # 코호트의 모든 기록이 no_circ 인 경우 제외

mimic_v2 = define_ambcirc(mimic_v1, 'mimic') # circ - amb - circ -> circ - amb_circ - circ
eicu_v2 = define_ambcirc(eicu_v1, 'eicu')

mimic_v3 = early_event_prediction_label(mimic_v2) # case 1, case 2 정의
eicu_v3 = early_event_prediction_label(eicu_v2)

mimic_v4 = optimized_recovered_labeler(mimic_v3, 'mimic') # case 3, case 4 정의
eicu_v4 = optimized_recovered_labeler(eicu_v3, 'eicu')

mimic_v5 = mimic_v4[~(mimic_v4['classes']=='undefined')]
eicu_v5 = eicu_v4[~(eicu_v4['classes']=='undefined')]

100%|██████████| 11969/11969 [08:27<00:00, 23.61it/s]


In [4]:
invalid_columns_mimic, _ = find_invalid_columns(mimic_v5)
invalid_columns_eicu, _ = find_invalid_columns(eicu_v5)

mimic_v6 = replace_inf_with_previous(mimic_v5, invalid_columns_mimic)
eicu_v6 = replace_inf_with_previous(eicu_v5, invalid_columns_eicu)

In [8]:
mimic_v7 = del_noise(mimic_v6, 'mimic')
eicu_v7 = del_noise(eicu_v6, 'eicu')

step1 모든 관측치가 amb ,0, 0인 stay 제거
step1 완료
step2 forward fill 잔재 제거(특정 시점부터 마지막까지 모두 amb-0-0인 경우)
step2 완료
step3 한번이라도 circulatory failure event가 발생하지 않은 stay 제거
step3 완료, 제거 stay 수 5804


In [11]:
def print_summary(dataset, dataset_name, stay_id_col, patient_id_col):
    print(f'{dataset_name} CACE DEFINED\n')
    print(f'전체 샘플의 개수: {len(dataset)}\n')
    print('샘플의 클래스 개수:')
    print(dataset['classes'].value_counts().sort_index())
    print(f'\n총 Stay 수: {len(dataset[stay_id_col].unique())}')
    print(f'\n총 환자 수: {len(dataset[patient_id_col].unique())}\n')

print_summary(mimic_v7, 'MIMIC-IV', 'stay_id', 'subject_id')

print_summary(eicu_v7, 'eICU', 'patientunitstayid', 'uniquepid')

MIMIC-IV CACE DEFINED

전체 샘플의 개수: 927484

샘플의 클래스 개수:
classes
0    612058
1     59562
2     16989
3    238875
Name: count, dtype: int64

총 Stay 수: 8883

총 환자 수: 8546

eICU CACE DEFINED

전체 샘플의 개수: 569246

샘플의 클래스 개수:
classes
0    378172
1     46104
2      6186
3    138784
Name: count, dtype: int64

총 Stay 수: 5208

총 환자 수: 5075



In [17]:
mimic_v7.drop(['Unnamed: 0'], axis = 1).to_csv('MIMIC-IV.csv.gz', index=False, compression='gzip')
eicu_v7.drop(['Unnamed: 0'], axis = 1).to_csv('eICU.csv.gz', index=False, compression='gzip')